#Setup

In [1]:
!nvidia-smi

Tue Sep  6 13:39:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q diffusers
!pip install -q transformers scipy ftfy
!pip install -q "ipywidgets>=7,<8"
!pip install -q gradio

     |████████████████████████████████| 112 kB 9.5 MB/s 
     |████████████████████████████████| 120 kB 54.9 MB/s 
     |████████████████████████████████| 4.7 MB 8.2 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 6.6 MB 51.0 MB/s 
     |████████████████████████████████| 1.6 MB 8.3 MB/s 
     |████████████████████████████████| 6.1 MB 8.1 MB/s 
     |████████████████████████████████| 270 kB 57.6 MB/s 
     |████████████████████████████████| 212 kB 62.7 MB/s 
     |████████████████████████████████| 57 kB 3.4 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 112 kB 72.8 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 2.3 MB 58.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |██████████████████████████

In [3]:
from google.colab import output
output.enable_custom_widget_manager()

from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [6]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", use_auth_token=True).to('cuda')

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/743 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

#Make images

In [33]:
from torch import autocast

def make_img(prompt, width=512, height=512, num_imgs=1):
    # prompts = [prompt] * num_imgs
    imgs = []
    for _ in range(num_imgs):
        images = pipeline(prompt, width=width, height=height)["sample"][0]
        imgs.append(images)
    return imgs

In [ ]:
import gradio as gr

with gr.Blocks() as frontend:
    with gr.Row():
        with gr.Column(scale=8):
            prompt = gr.Textbox(label="Prompt") 
        with gr.Column(scale=1):
            diffuse = gr.Button(value='Dream', label="Diffuse")
    with gr.Row():
        height = gr.Slider(512, 1280, 512, step=64, label="Height", interactive=True)
        width = gr.Slider(minimum=512, maximum=1280, value=512, step=64, label="Width", interactive=True)
        num_imgs = gr.Slider(minimum=1, maximum=10, value=3, step=1, label="Number of Images", interactive=True)
    gallery = gr.Gallery(label="Output", elem_id='Images')# .style(grid=[4,4])
    diffuse.click(fn=make_img, inputs=[prompt, height, width, num_imgs], outputs=gallery)

frontend.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://25732.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Exception in thread Thread-253:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/gradio/tunneling.py", line 51, in reverse_forward_tunnel
    transport.request_port_forward("", server_port)
  File "/usr/local/lib/python3.7/dist-packages/paramiko/transport.py", line 1104, in request_port_forward
    raise SSHException("TCP forwarding request denied")
paramiko.ssh_exception.SSHException: TCP forwarding request denied



In [ ]:
from torch import autocast
import IPython.display
prompt = "space mining colony by Alena Aenami and Anato Finnstark "#@param{type:"string"}
num_imgs = 5#@param{type:"integer"}
height = 512#@param{type:"integer"}
width = 512#@param{type:"integer"}

for i in range(num_imgs):
    with autocast("cuda"):
        image = pipeline(prompt, width=width, height=height)["sample"][0]
    IPython.display.display(image)